# Happy Portfolio Analysis

In [1]:
# Import relevant libraries

import pickle
import pandas as pd
import plotly.express as px
import numpy as np
import plotly.graph_objects as go


In [2]:
# Read in etf data from pkl file

pkl_file = open('etf_df.pkl', 'rb')
dataframe2 = pickle.load(pkl_file)
pkl_file.close()

# load etf data back into Dataframe
all_etf_df = pd.DataFrame(dataframe2)

# Convert month end prices to monthly percentage change and drop na values

all_etf_df = all_etf_df.pct_change().dropna()


In [3]:
# Review dataframe

 all_etf_df.head()

In [4]:
# Read in country data from pkl file

pkl_file = open('wh_2015_2019.pkl', 'rb')
dataframe3 = pickle.load(pkl_file)
pkl_file.close()

# load etf data back into Dataframe
wh_2015_2019_df = pd.DataFrame(dataframe3)

# Drop Year Column
wh_2015_2019_df = wh_2015_2019_df.drop(columns=['Year'])

UnpicklingError: invalid load key, '\xe2'.

In [ ]:
# Review Dataframe 

wh_2015_2019_df

In [ ]:
# Transpose dataframe to allow mapping to ETFs by Country

wh_2015_2019_df = wh_2015_2019_df.transpose()

wh_2015_2019_df


## Portfolio Mapping

In [ ]:
# Create Map of ETFs to Countries

etf_country_map = {
    "Finland":"iShares MSCI Finland Capped",
    "Denmark":"iShares MSCI Denmark Capped",
    "Norway":"iShares MSCI Norway Capped",
    "Netherlands":"iShares MSCI Netherlands",
    "Switzerland":"iShares MSCI Switzerland Capped",
    "Sweden":"iShares MSCI Sweden Capped",
    "New Zealand":"iShares MSCI New Zealand Capped",
    "Canada":"iShares MSCI Canada",
    "Austria":"iShares MSCI Austria Capped",
    "Australia":"iShares MSCI Australia",
    "Israel":"iShares MSCI Israel Capped",
    "United Kingdom":"iShares MSCI United Kingdom",
    "United States":"ishares S&P 500",
    "Ireland":"iShares MSCI Ireland",
    "Germany":"iShares Currency Hedged MSCI Germany",
    "Mexico":"iShares MSCI Mexico Capped",
    "Benchmark":"iShares MSCI World"
}

# Create DataFrame of Countries and Corresponding ETFs

etf_country_map_df = pd.DataFrame.from_dict(etf_country_map, orient='index')
etf_country_map_df.columns = ["ETF"]
etf_country_map_df.index.name = "Country"
etf_country_map_df

In [ ]:
# Combine Dataframes to show Happiness Score by ETF (for Portfolio Weights)

country_etf_combined = pd.merge(etf_country_map_df, wh_2015_2019_df,on='Country', how='outer')

country_etf_combined.set_index('ETF', inplace=True, drop=True)
columns = [2015,2016,2017,2018,2019]
country_etf_combined.columns = columns

country_etf_combined

# Portfolio Weightings

In [ ]:
# Create new Dataframe to weight the ETFs by Happiness Score
country_etf_weighted = country_etf_combined


# Recalculate Dataframe to Weight each Score by Year

country_etf_weighted["2015 Weights"] = (country_etf_weighted[2015]/country_etf_weighted[2015].sum()) * 100
country_etf_weighted["2016 Weights"] = (country_etf_weighted[2016]/country_etf_weighted[2016].sum()) * 100
country_etf_weighted["2017 Weights"] = (country_etf_weighted[2017]/country_etf_weighted[2017].sum()) * 100
country_etf_weighted["2018 Weights"] = (country_etf_weighted[2018]/country_etf_weighted[2018].sum()) * 100
country_etf_weighted["2019 Weights"] = (country_etf_weighted[2019]/country_etf_weighted[2019].sum()) * 100

# Drop original score columns to leave only the portfolio weights
country_etf_weighted = country_etf_weighted.drop(columns=[2015,2016,2017,2018,2019])

# Rename the columns
columns = [2015,2016,2017,2018,2019]
country_etf_weighted.columns = columns

# Review the weighted DataFrame
country_etf_weighted


In [ ]:
# Check all columns total 100
country_etf_weighted.sum()

In [ ]:
donut_2015 = pd.DataFrame(country_etf_weighted[2015].dropna())
donut_2015

In [ ]:
# Use `hole` to create a donut-like pie chart

names = [
    "Finland", 
    "Denmark", 
    "Norway", 
    "Netherlands",
    "Switzerland",
    "Sweden",
    "New Zealand",
    "Canada",
    "Austria",
    "Australia",
    "Israel",
    "United States",
    "Mexico"]

fig = px.pie(
    donut_2015, 
    values=2015, 
    names=names, 
    title='Porfolio Weights', 
    hole=0.5,
    hover_data=[2015],
)

fig.update_traces(
    textposition='outside',
    textinfo='label'
)

for template in ["plotly_dark"]:
    fig.update_layout(template=template, annotations=[dict(text='2015', x=0.5, y=0.5, font_size=20, showarrow=False)])

fig.show()


## Portfolio Return

### 2015

In [ ]:

# Set starting values for 2015 based on weight
portfolio_start_2015 = country_etf_weighted.iloc[:,0]

# Check total = 100
display(portfolio_start_2015.sum())

# Create new dataframe for 2015 return calc using the 2015 start values
portfolio_return_2015 = pd.DataFrame(portfolio_start_2015)
portfolio_return_2015 = portfolio_return_2015.transpose()

# Rename weighted value index to start date 01-01-2015
portfolio_return_2015 = portfolio_return_2015.reset_index(drop=True)
portfolio_return_2015 = portfolio_return_2015.rename(index={0:'2015-01-01'})

# Append the all_etf_df monthly returns
portfolio_return_2015 = portfolio_return_2015.append(all_etf_df.loc['2015-02-01':'2015-12-31']).sort_index()

# Calculate cumulative return and drop na row
portfolio_return_2015 = (1 + portfolio_return_2015.shift(1)).cumprod().dropna(how='all')

# Drop ETFs not included in portfolio for 2015
portfolio_return_2015 = portfolio_return_2015.dropna(axis=1)

# Review the dataframe
portfolio_return_2015

In [ ]:
# Create a new benchmark dataframe, rebase to 100 for comparible data to portfolio

benchmark = pd.DataFrame(index=["Benchmark"], data=[100], columns=["2015-01-01"])

# Append benchmark data from all_etf_df and combine into a single columns

benchmark = benchmark.append(all_etf_df["iShares MSCI World"])
benchmark = benchmark.transpose().sum(axis=1)

# Slice by date 
benchmark = benchmark.loc["2015-01-01":"2015-12-31"]

# Calculate cumulative return and drop na (first value)
benchmark = (1 + benchmark.shift(1)).cumprod().dropna()
benchmark

In [ ]:
# Combine return for portfolio

portfolio_return_2015 = portfolio_return_2015.sum(axis=1)
portfolio_return_2015

In [ ]:
# Create plot dataframe by combining benchmark and portfolio return dfs

plot_df = pd.concat([portfolio_return_2015, benchmark.loc["2015-01-01":"2015-12-31"]], axis=1)

# Rename columns
columns = ["Happy Portfolio","Benchmark"]
plot_df.columns = columns

# Add back in the start value of 100 for the plot
new_row = pd.DataFrame({"Happy Portfolio":100, "Benchmark":100}, index=['2015-01-01'])
plot_df = pd.concat([plot_df, new_row]).sort_index()

# Review
plot_df


In [ ]:
# Plot 2015

fig = px.line(
    plot_df,
    title="2015",
    height=500,
    width=1000,
    labels={
        "value":"Return",
        "index":"Date",
        "variable":"Happy Portfolio"
    }
)
fig.show()

### 2016

In [ ]:
# Set starting values for 2016 based on 2015 year end value and 2016 weights
portfolio_start_2016 = portfolio_return_2015[-1] / 100 * country_etf_weighted.iloc[:,1]

# Check total = 2015 year end value $111.69
display(portfolio_start_2016.sum())

# Create new dataframe for 2016 return calc using the 2016 start values
portfolio_return_2016 = pd.DataFrame(index=["2015-12-31"],data=[portfolio_start_2016])

# # Rename weighted value index to start date 01-01-2015
# portfolio_return_2015 = portfolio_return_2015.reset_index(drop=True)
# portfolio_return_2015 = portfolio_return_2015.rename(index={0:'2015-01-01'})

# Append the all_etf_df monthly returns
portfolio_return_2016 = portfolio_return_2016.append(all_etf_df.loc['2016-01-01':'2016-12-31']).sort_index()

# Calculate cumulative return and drop na row
portfolio_return_2016 = (1 + portfolio_return_2016.shift(1)).cumprod().dropna(how='all')

# Drop ETFs not included in portfolio for 2015
portfolio_return_2016 = portfolio_return_2016.dropna(axis=1)

# Review the dataframe
portfolio_return_2016

In [ ]:
# Extend benchmark to 2016 - once all consolidated use one code for Benchmark

# Create a new benchmark dataframe, rebase to 100 for comparible data to portfolio

benchmark = pd.DataFrame(index=["Benchmark"], data=[100], columns=["2015-01-01"])

# Append benchmark data from all_etf_df and combine into a single columns

benchmark = benchmark.append(all_etf_df["iShares MSCI World"])
benchmark = benchmark.transpose().sum(axis=1)

# Slice by date 
benchmark = benchmark.loc["2015-01-01":"2016-12-31"]

# Calculate cumulative return and drop na (first value)
benchmark = (1 + benchmark.shift(1)).cumprod().dropna()
benchmark

In [ ]:
# Combine return for portfolio

portfolio_return_2016 = portfolio_return_2016.sum(axis=1)
portfolio_return_2016

In [ ]:
# Create plot dataframe by combining benchmark and portfolio return dfs

plot_df_2016 = pd.concat([portfolio_return_2015, portfolio_return_2016], axis=0)

plot_df_2016 = pd.concat([plot_df_2016, benchmark.loc["2015-01-01":"2016-12-31"]], axis=1)

# Rename columns
columns = ["Happy Portfolio","Benchmark"]
plot_df_2016.columns = columns

# Add back in the start value of 100 for the plot
new_row = pd.DataFrame({"Happy Portfolio":100, "Benchmark":100}, index=['2015-01-01'])
plot_df_2016 = pd.concat([plot_df_2016, new_row]).sort_index()

# Review
plot_df_2016

In [ ]:
# Plot 2015-2016

fig = px.line(
    plot_df_2016,
    title="2015-2016",
    height=500,
    width=1000,
    labels={
        "value":"Return",
        "index":"Date",
        "variable":"Happy Portfolio"
    }
)
fig.show()

## 2017

In [ ]:
# Set starting values for 2017 based on 2016 year end value and 2017 weights
portfolio_start_2017 = portfolio_return_2016[-1] / 100 * country_etf_weighted.iloc[:,2]

# Check total = 2016 year end value $126.55
display(portfolio_start_2017.sum())

# Create new dataframe for 2017 return calc using the 2017 start values
portfolio_return_2017 = pd.DataFrame(index=["2016-12-31"],data=[portfolio_start_2017])

# # Rename weighted value index to start date 01-01-2015
# portfolio_return_2015 = portfolio_return_2015.reset_index(drop=True)
# portfolio_return_2015 = portfolio_return_2015.rename(index={0:'2015-01-01'})

# Append the all_etf_df monthly returns
portfolio_return_2017 = portfolio_return_2017.append(all_etf_df.loc['2017-01-01':'2017-12-31']).sort_index()

# Calculate cumulative return and drop na row
portfolio_return_2017 = (1 + portfolio_return_2017.shift(1)).cumprod().dropna(how='all')

# Drop ETFs not included in portfolio for 2015
portfolio_return_2017 = portfolio_return_2017.dropna(axis=1)

# Review the dataframe
portfolio_return_2017

In [ ]:
# Extend benchmark to 2017 - once all consolidated use one code for Benchmark

# Create a new benchmark dataframe, rebase to 100 for comparible data to portfolio

benchmark = pd.DataFrame(index=["Benchmark"], data=[100], columns=["2015-01-01"])

# Append benchmark data from all_etf_df and combine into a single columns

benchmark = benchmark.append(all_etf_df["iShares MSCI World"])
benchmark = benchmark.transpose().sum(axis=1)

# Slice by date 
benchmark = benchmark.loc["2015-01-01":"2017-12-31"]

# Calculate cumulative return and drop na (first value)
benchmark = (1 + benchmark.shift(1)).cumprod().dropna()
benchmark

In [ ]:
# Combine return for portfolio

portfolio_return_2017 = portfolio_return_2017.sum(axis=1)
portfolio_return_2017

In [ ]:
# Create plot dataframe by combining benchmark and portfolio return dfs

plot_df_2017 = pd.concat([portfolio_return_2015, portfolio_return_2016, portfolio_return_2017], axis=0)

plot_df_2017 = pd.concat([plot_df_2017, benchmark.loc["2015-01-01":"2017-12-31"]], axis=1)

# Rename columns
columns = ["Happy Portfolio","Benchmark"]
plot_df_2017.columns = columns

# Add back in the start value of 100 for the plot
new_row = pd.DataFrame({"Happy Portfolio":100, "Benchmark":100}, index=['2015-01-01'])
plot_df_2017 = pd.concat([plot_df_2017, new_row]).sort_index()

# Review
plot_df_2017

In [ ]:
# Plot 2015-2017

fig = px.line(
    plot_df_2017,
    title="2015-2017",
    height=500,
    width=1000,
    labels={
        "value":"Return",
        "index":"Date",
        "variable":"Happy Portfolio"
    }
)
fig.show()

## 2018

In [ ]:
# Set starting values for 2018 based on 2017 year end value and 2018 weights
portfolio_start_2018 = portfolio_return_2017[-1] / 100 * country_etf_weighted.iloc[:,3]

# Check total = 2017 year end value $168.07
display(portfolio_start_2018.sum())

# Create new dataframe for 2018 return calc using the 2018 start values
portfolio_return_2018 = pd.DataFrame(index=["2017-12-31"],data=[portfolio_start_2018])

# # Rename weighted value index to start date 01-01-2015
# portfolio_return_2015 = portfolio_return_2015.reset_index(drop=True)
# portfolio_return_2015 = portfolio_return_2015.rename(index={0:'2015-01-01'})

# Append the all_etf_df monthly returns
portfolio_return_2018 = portfolio_return_2018.append(all_etf_df.loc['2018-01-01':'2018-12-31']).sort_index()

# Calculate cumulative return and drop na row
portfolio_return_2018 = (1 + portfolio_return_2018.shift(1)).cumprod().dropna(how='all')

# Drop ETFs not included in portfolio for 2018
portfolio_return_2018 = portfolio_return_2018.dropna(axis=1)

# Review the dataframe
portfolio_return_2018

In [ ]:
# Extend benchmark to 2018 - once all consolidated use one code for Benchmark

# Create a new benchmark dataframe, rebase to 100 for comparible data to portfolio

benchmark = pd.DataFrame(index=["Benchmark"], data=[100], columns=["2015-01-01"])

# Append benchmark data from all_etf_df and combine into a single columns

benchmark = benchmark.append(all_etf_df["iShares MSCI World"])
benchmark = benchmark.transpose().sum(axis=1)

# Slice by date 
benchmark = benchmark.loc["2015-01-01":"2018-12-31"]

# Calculate cumulative return and drop na (first value)
benchmark = (1 + benchmark.shift(1)).cumprod().dropna()
benchmark

In [ ]:
# Combine return for portfolio

portfolio_return_2018 = portfolio_return_2018.sum(axis=1)
portfolio_return_2018

In [ ]:
# Create plot dataframe by combining benchmark and portfolio return dfs

plot_df_2018 = pd.concat([portfolio_return_2015, portfolio_return_2016, portfolio_return_2017, portfolio_return_2018], axis=0)

plot_df_2018 = pd.concat([plot_df_2018, benchmark.loc["2015-01-01":"2018-12-31"]], axis=1)

# Rename columns
columns = ["Happy Portfolio","Benchmark"]
plot_df_2018.columns = columns

# Add back in the start value of 100 for the plot
new_row = pd.DataFrame({"Happy Portfolio":100, "Benchmark":100}, index=['2015-01-01'])
plot_df_2018 = pd.concat([plot_df_2018, new_row]).sort_index()

# Review
plot_df_2018

In [ ]:
# Plot 2015-2018

fig = px.line(
    plot_df_2018,
    title="Growth of $100 invested 2015-2018",
    height=500,
    width=1000,
    labels={
        "value":"Return",
        "index":"Date",
        "variable":"Happy Portfolio"
    }
)
fig.show()

## 2019

In [ ]:
# Set starting values for 2019 based on 2018 year end value and 2019 weights
portfolio_start_2019 = portfolio_return_2018[-1] / 100 * country_etf_weighted.iloc[:,3]

# Check total = 2018 year end value $162.80
display(portfolio_start_2019.sum())

# Create new dataframe for 2019 return calc using the 2019 start values
portfolio_return_2019 = pd.DataFrame(index=["2018-12-31"],data=[portfolio_start_2019])

# # Rename weighted value index to start date 01-01-2015
# portfolio_return_2015 = portfolio_return_2015.reset_index(drop=True)
# portfolio_return_2015 = portfolio_return_2015.rename(index={0:'2015-01-01'})

# Append the all_etf_df monthly returns
portfolio_return_2019 = portfolio_return_2019.append(all_etf_df.loc['2019-01-01':'2019-12-31']).sort_index()

# Calculate cumulative return and drop na row
portfolio_return_2019 = (1 + portfolio_return_2019.shift(1)).cumprod().dropna(how='all')

# Drop ETFs not included in portfolio for 2019
portfolio_return_2019 = portfolio_return_2019.dropna(axis=1)

# Review the dataframe
portfolio_return_2019

In [ ]:
# Extend benchmark to 2019 - once all consolidated use one code for Benchmark

# Create a new benchmark dataframe, rebase to 100 for comparible data to portfolio

benchmark = pd.DataFrame(index=["Benchmark"], data=[100], columns=["2015-01-01"])

# Append benchmark data from all_etf_df and combine into a single columns

benchmark = benchmark.append(all_etf_df["iShares MSCI World"])
benchmark = benchmark.transpose().sum(axis=1)

# Slice by date 
benchmark = benchmark.loc["2015-01-01":"2019-12-31"]

# Calculate cumulative return and drop na (first value)
benchmark = (1 + benchmark.shift(1)).cumprod().dropna()
benchmark

In [ ]:
# Combine return for portfolio

portfolio_return_2019 = portfolio_return_2019.sum(axis=1)
portfolio_return_2019

In [ ]:
# Create plot dataframe by combining benchmark and portfolio return dfs

plot_df_2019 = pd.concat([portfolio_return_2015, portfolio_return_2016, portfolio_return_2017, portfolio_return_2018, portfolio_return_2019], axis=0)

plot_df_2019 = pd.concat([plot_df_2019, benchmark.loc["2015-01-01":"2019-12-31"]], axis=1)

# Rename columns
columns = ["Happy Portfolio","Benchmark"]
plot_df_2019.columns = columns

# Add back in the start value of 100 for the plot
new_row = pd.DataFrame({"Happy Portfolio":100, "Benchmark":100}, index=['2015-01-01'])
plot_df_2019 = pd.concat([plot_df_2019, new_row]).sort_index()

# Review
plot_df_2019

In [ ]:
# Plot 2015-2019

fig = px.line(
    plot_df_2019,
    title="Growth of $100 invested 2015-2019",
    height=500,
    width=1000,
    labels={
        "value":"Return",
        "index":"Date",
        "variable":"Happy Portfolio"
    }
)

for template in ["plotly_dark"]:
    fig.update_layout(template=template)

fig.show()

## Review calcs needed from here

In [ ]:
# Annual Return
all_etf_df_return_2015 = all_etf_df.loc['2015-01-01':'2015-12-31'].mean() * 12
all_etf_df_return_2015 = pd.DataFrame(all_etf_df_return_2015, columns=[2015])

all_etf_df_return_2016 = all_etf_df.loc['2016-01-01':'2016-12-31'].mean() * 12
all_etf_df_return_2016 = pd.DataFrame(all_etf_df_return_2016, columns=[2016])

all_etf_df_return_2017 = all_etf_df.loc['2017-01-01':'2017-12-31'].mean() * 12
all_etf_df_return_2017 = pd.DataFrame(all_etf_df_return_2017, columns=[2017])

all_etf_df_return_2018 = all_etf_df.loc['2018-01-01':'2018-12-31'].mean() * 12
all_etf_df_return_2018 = pd.DataFrame(all_etf_df_return_2018, columns=[2018])

all_etf_df_return_2019 = all_etf_df.loc['2019-01-01':'2019-12-31'].mean() * 12
all_etf_df_return_2019 = pd.DataFrame(all_etf_df_return_2019, columns=[2019])

etf_annual_returns = pd.concat([all_etf_df_return_2015, all_etf_df_return_2016, all_etf_df_return_2017, all_etf_df_return_2018, all_etf_df_return_2019], axis=1)

etf_annual_returns

## Monthly Portfolio Return vs Benchmark

In [ ]:
# Empty dataframe to collect portfolio weights from below for loop
portfolio_2019_df = pd.DataFrame()

# Return * Portfolio Weight
for etf in portfolio_weight_2019:
    etf_column = all_etf_df[etf] * portfolio_weight_2019[etf]
    portfolio_2019_df[etf] = etf_column

# Filter Dates
portfolio_2019_df = portfolio_2019_df.loc['2019-01-01':'2019-12-31']
    
# Combined return for all funds and rename Happy Portfolio
portfolio_2019_df = pd.DataFrame(portfolio_2019_df.sum(axis=1), columns=['Happy Portfolio'])

# Add Benchmark
benchmark = "iShares MSCI World"

benchmark_return = pd.DataFrame(all_etf_df[benchmark])

# slice by date
benchmark_return = benchmark_return.loc["2019-01-01":"2019-12-31"]

portfolio_2019_df = pd.concat([portfolio_2019_df,benchmark_return], axis=1)
portfolio_2019_df

## Cumulative Portfolio Return vs Benchmark

In [ ]:
# cumulative returns
cum_portfolio_2019 = (1 + portfolio_2019_df).cumprod()
cum_portfolio_2019


In [ ]:
# Plot 2019 vs benchmark

fig = px.line(
    cum_portfolio_2019,
    title="Cumulative Return of Happy Portfolio vs Benchmark 2019",
    height=500,
    width=1000,
    labels={
        "value":"Return",
        "date":"Date",
        "variable":""
    }
)
fig.show()

In [ ]:
## Portfolio and Benchmark Statistics

In [ ]:
portfolio_2019_df.describe()

In [ ]:
fig = px.box(
    portfolio_2019_df,
    title='Box Plot of Happy Portfolio and Benchmark Returns'
)
fig.show()

In [ ]:
# Sharpe Ratio

average_annual_return = portfolio_2019_df.mean() * 12
display(average_annual_return)

annual_sd_portfolio = portfolio_2019_df.std() * np.sqrt(12)
display(annual_sd_portfolio)

sharpe_ratios = average_annual_return / annual_sd_portfolio
display(sharpe_ratios)

fig = px.bar(
    sharpe_ratios,
    title="Sharpe Ratios of Happy Fund and Benchmark"
)
fig.show()

In [ ]:
# PULL IN WORLD HAPPINESS DF TO OBTAIN RANKINGS OR SCORE
# CREATE WEIGHTINGS VARIABLES PER YEAR
# CREATE PERFORMANCE VARIABLES BY YEAR BASED ON WEIGHTINGS AND RANKINGS 

